In [1]:
%matplotlib inline
import math
import numpy as np
from numpy import *
from scipy.optimize import curve_fit
from scipy import signal
from os import path
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Input, Dense, GRU, Embedding, Dense, Activation, Dropout, LSTM
from keras.optimizers import Adam
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.models import load_model


Using TensorFlow backend.


In [2]:
plt.rc('text', usetex=True)

outpath = "../../../plots/temperatures"
inpath = "../../../"

currentfile = "Imitator_2_2400.csv"

# Read from file
strdatatype = np.dtype([('N', np.int_, (2,)), ('Time_Count', np.int_ ), ('Mode', np.int_ ),
                        ('T', np.float_, (10,)), ('S', np.bool_, (10,)), ('System_State', np.bool_ )])
N, Time_Count, Mode, T, S, System_State = np.loadtxt(path.join(inpath, currentfile), 
                                                     unpack=True, delimiter=';', skiprows=1, dtype=strdatatype)

In [3]:
max_possible_model_temperature = 15.0
min_possible_model_temperature = 70.0
def scale_T(_T):
    return (_T - min_possible_model_temperature) / (max_possible_model_temperature - min_possible_model_temperature)

def unscale_T(_T):
    return (_T) * (max_possible_model_temperature - min_possible_model_temperature) + min_possible_model_temperature

In [4]:
sT = scale_T(T)

In [5]:
print("Min:", np.min(sT))
print("Max:", np.max(sT))

Min: 0.0883266748985012
Max: 0.9380161784282788


In [6]:
delta = np.amin(np.abs(sT[:-2, :] - sT[1:-1, :])) 
delta

2.1317509536489254e-06

In [7]:
#data augmintation -- 50000 slightly tuned copies of T
agmntCount=5
agmntdT=np.zeros((agmntCount, np.size(sT[:,0]), np.size(sT[0,:])))
agmntdT[0,:,:] = sT
np.random.seed(0)
mu, sigma = 0, delta*100
for i in range(1, agmntCount):
    agmntdT[i] = agmntdT[0] + np.random.normal(mu, sigma, (np.size(sT[:,0]), np.size(sT[0,:])))

In [8]:
predictSteps = 1
#Keep dataset tail for validate prediction quality
cutFromTail = 60
#In order to have shifted and unshifted series with same shape
t = cutFromTail + predictSteps

In [9]:
inData = np.zeros((agmntCount, np.size(sT[:-t,0]), np.size(sT[0,:]) + 1))
for i in range(0, agmntCount):
    inData[i,:,0] = Mode[:-t] / np.amax(Mode[:-t])
    inData[i,:,1:]= agmntdT[i,:-t,:]

In [10]:
num_data = len(inData[0,:,0])
train_split = 0.80
num_train = int(train_split * num_data)
num_test = num_data - num_train
inData_train = inData[:, 0:num_train, :]
inData_test = inData[:, num_train:, :]
num_inData_signals = inData.shape[2]

In [11]:
model = Sequential()

In [13]:
outputBlockId = 0
path_checkpoint = str(outputBlockId)+'_checkpoint_test.keras'
try:
    model = load_model(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

Error trying to load checkpoint.
Cannot create group in read-only mode.
